In [1]:
from instruments import Voltmeter,Sourcemeter,VSourcemeter,MercuryiTC,MercuryiPS,Lakeshore
from instrument_group import InstrumentGroup
import pyvisa
from time import sleep
import numpy as np

# See available instruments

In [3]:
rm = pyvisa.ResourceManager()
inst = rm.list_resources()
print(inst)

# Initialize instruments and create the instrument group

In [ ]:
# Initialise all instruments and create the instrument group.
# Add instruments you need, comment out instruments not being used.

IG = InstrumentGroup(
    voltmeters = [
        ("A",Voltmeter('GPIB0::6::INSTR')),
        ("B",Voltmeter('GPIB0::12::INSTR')),
        ("C",Voltmeter('GPIB0::22::INSTR')),
        ("D",Voltmeter('GPIB0::25::INSTR')),
        ],
    sourcemeters = [
        ("A",Sourcemeter('GPIB0::5::INSTR')),
        ("B",Sourcemeter('GPIB0::11::INSTR')),
        ],
    Vsourcemeters = [
        ("A",VSourcemeter('GPIB0::23::INSTR')),
        ("B",VSourcemeter('GPIB0::24::INSTR')),
        ],
    iTC = MercuryiTC('ASRL7::INSTR'),
    iPS = MercuryiPS('ASRL8::INSTR'),
    lakeshore = Lakeshore('GPIB0::26::INSTR'),
    )

# Valid commands:

### Measurements

Measure continuously
```python
    IG.measure_until_interrupted(filename,timeout_hours=12)
```

Ramp temperature. Controller is 'VTI', 'probe', or 'both', Ts are in K, rate is in K/min  
```python
    IG.ramp_T(filename,controller,Ts,rates,threshold=0.05,base_T_threshold=0.005,timeout_hours=12)
```

Set temperature. Controller is 'VTI', 'probe', or 'both', T is in K  
```python
    IG.set_T(filename,controller,T,threshold=0.05,base_T_threshold=0.005,timeout_hours=12)
```

Ramp magnetic field. Bs are in T, rate is in T/min  
```python
    IG.ramp_B(filename,Bs,rates,threshold=0.005,timeout_hours=12)
```

Set backgate. Vgs are in V, compliance is in A  
```python
    IG.set_Vg(filename,Vgs,compliance=5e-7,wait=0.1)
```

Set current and measure, Is are in A, compliance is in V  
```python
    IG.perform_IV(filename,Is,compliance=5,wait=0.1)
```

### Others

Wait
```python
    sleep(seconds)
```

Set current, I is in A, compliance is in V  
```python
    IG.set_current(I,compliance=5)
```

# Run your script

An example is shown below

In [ ]:
folder = "my_folder/"
IG.set_comment("A description of my sample.")

# perform a log spaced IV
IG.set_filename(folder+"my_sample_IV_roomT.txt")
Is = np.logspace(-9,-5,100)
IG.perform_IV(Is,compliance=5,wait=0.1)

# cooldown to base temperature
IG.set_filename(folder+"my_sample_cooldown.txt")
IG.set_current(1e-6)
IG.set_T("both",0)

# perform an IV at base temperature
IG.set_filename(folder+"my_sample_IV_baseT.txt")
Is = np.logspace(-9,-5,100)
IG.perform_IV(Is,compliance=5,wait=0.1)

# set the temperature to 5K
IG.dont_measure() # we don't need to record this part
IG.set_T("both",5)
sleep(600) # wait 10 minutes

# sweep the gate voltage back and forth
IG.set_current(1e-6)
Vgs=np.concatenate([
    np.linspace(0,-50,51),
    np.linspace(-50,50,101),
    np.linspace(50,-100,151),
    np.linspace(-100,100,201),
    np.linspace(100,-150,251),
    np.linspace(-150,150,301),
    np.linspace(150,-200,351),
    np.linspace(-200,200,401),
    np.linspace(200,-200,401),
    np.linspace(-200,200,401),
    np.linspace(200,-200,401),
    -200*np.ones(500)
])
IG.set_filename(folder+"my_sample_backgate-sweep.txt") # this automatically turns measurment mode back on
IG.set_Vg(Vgs)

# setpoints for the VTI heater
VTI_heater_setpoints=np.concatenate([
                                     0*np.ones(100),
                                     np.linspace(0,0.8,800), 
                                     0.8*np.ones(100), 
                                     np.linspace(0.8,0,800),
                                      0*np.ones(100),
                                   ])

# go to different gate voltages, perform a temperature sweep and magnetic field sweep
for Vg in [-200,-150,-100,-50,0,50,100,150,200]:
    IG.dont_measure()
    IG.set_Vg(Vg)
    sleep(1800)

    # Do a temperature sweep with heater directly
    IG.set_filename(folder+f"my_sample_RT_Vg={Vg}V.txt")
    IG.ramp_heater(0,VTI_heater_setpoints)

    sleep(120)
    
    # Sweep the magnetic field
    IG.set_filename(folder+f"my_sample_RB_Vg={Vg}V.txt")
    IG.ramp_B([12,-12,0],rates=0.3)